# Ejercicio 1 - Oil & Gas

Importamos pandas y numpy

In [21]:
import pandas as pd
import numpy as np

## Lectura de Datos

In [20]:
df_lectura = pd.read_excel("oilgascounty.xls","oilgascounty")
df_lectura

,FIPS,geoid,Stabr,County_Name,Rural_Urban_Continuum_Code_2013,Urban_Influence_2013,Metro_Nonmetro_2013,Metro_Micro_Noncore_2013,oil2000,oil2001,oil2002,oil2003,oil2004,oil2005,oil2006,oil2007,oil2008,oil2009,oil2010,oil2011,gas2000,gas2001,gas2002,gas2003,gas2004,gas2005,gas2006,gas2007,gas2008,gas2009,gas2010,gas2011,oil_change_group,gas_change_group,oil_gas_change_group
0,1001,1001,AL,Autauga County,2,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Status Quo,Status Quo,Status Quo
1,1003,1003,AL,Baldwin County,3,2,1,2,138072,134666,138011,127985,130763,118043,103992,112303,97623,84982,101955,94638,72543902,98699994,107142655,101510068,90146850,84536875,83951640,82876786,78547145,68525628,63069025,51041072,Status Quo,H_Decline,H_Decline
2,1005,1005,AL,Barbour County,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Status Quo,Status Quo,Status Quo
3,1007,1007,AL,Bibb County,1,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8301,98853,480015,684143,551719,453132,400504,Status Quo,Status Quo,Status Quo
4,1009,1009,AL,Blount County,1,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20516,61054,3594,21496,Status Quo,Status Quo,Status Quo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,56037,56037,WY,Sweetwater County,5,8,0,1,4429736,4482060,4474724,4472827,4520293,4874112,5294981,5616600,5451750,5122748,7443009,6003288,233453423,227839854,229597363,237123055,233475073,224874681,238160010,235399758,241096991,230934287,244962845,246392154,H_Growth,H_Growth,H_Growth
3105,56039,56039,WY,Teton County,7,8,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Status Quo,Status Quo,Status Quo
3106,56041,56041,WY,Uinta County,7,8,0,1,4437208,3826577,3079892,2936254,2600057,2226741,1911678,1509269,1345736,1126867,1097838,982710,195116690,183751252,170507572,166789181,151313197,141482273,137449416,126075100,127396828,106936672,118581276,106114215,H_Decline,H_Decline,H_Decline
3107,56043,56043,WY,Washakie County,7,11,0,0,1417992,1213574,1049308,976145,899743,765225,780944,787011,774921,729361,720502,648727,5347192,5273481,4619003,4036863,3903919,3525292,3177370,2851401,2576087,2416769,2252865,1961277,H_Decline,Status Quo,H_Decline


## Exploración del dataset

In [22]:
df_lectura.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3109 entries, 0 to 3108
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   FIPS                             3109 non-null   int64 
 1   geoid                            3109 non-null   int64 
 2   Stabr                            3109 non-null   object
 3   County_Name                      3109 non-null   object
 4   Rural_Urban_Continuum_Code_2013  3109 non-null   int64 
 5   Urban_Influence_2013             3109 non-null   int64 
 6   Metro_Nonmetro_2013              3109 non-null   int64 
 7   Metro_Micro_Noncore_2013         3109 non-null   int64 
 8   oil2000                          3109 non-null   int64 
 9   oil2001                          3109 non-null   int64 
 10  oil2002                          3109 non-null   int64 
 11  oil2003                          3109 non-null   int64 
 12  oil2004                          3

## Limpieza de los datos

Nos cargamos las columnas que sobran

In [23]:
df_1 = df_lectura \
        .drop (columns = ["gas_change_group","Urban_Influence_2013","Metro_Nonmetro_2013","Rural_Urban_Continuum_Code_2013","Metro_Micro_Noncore_2013","oil_change_group","oil_gas_change_group"]) \
        .melt(id_vars = ["FIPS","geoid","Stabr","County_Name"],var_name= "Product_year",value_name="Value")
df_1

,FIPS,geoid,Stabr,County_Name,Product_year,Value
0,1001,1001,AL,Autauga County,oil2000,0
1,1003,1003,AL,Baldwin County,oil2000,138072
2,1005,1005,AL,Barbour County,oil2000,0
3,1007,1007,AL,Bibb County,oil2000,0
4,1009,1009,AL,Blount County,oil2000,0
...,...,...,...,...,...,...
74611,56037,56037,WY,Sweetwater County,gas2011,246392154
74612,56039,56039,WY,Teton County,gas2011,0
74613,56041,56041,WY,Uinta County,gas2011,106114215
74614,56043,56043,WY,Washakie County,gas2011,1961277


Usamos melt para poner la tabla con las cabeceras bien

Separamos Producto de año

In [24]:
df_2 = df_1.assign (
        Product = lambda dataset: dataset["Product_year"].map(lambda Value : Value[:3]),
        Year = lambda dataset: dataset["Product_year"].map(lambda Value : int(Value[-4:]))
        ).drop( columns="Product_year")
df_2 = df_2[df_2['Value'] !=0]
df_2

,FIPS,geoid,Stabr,County_Name,Value,Product,Year
1,1003,1003,AL,Baldwin County,138072,oil,2000
11,1023,1023,AL,Choctaw County,652067,oil,2000
12,1025,1025,AL,Clarke County,330933,oil,2000
17,1035,1035,AL,Conecuh County,110132,oil,2000
19,1039,1039,AL,Covington County,393178,oil,2000
...,...,...,...,...,...,...,...
74610,56035,56035,WY,Sublette County,1132276281,gas,2011
74611,56037,56037,WY,Sweetwater County,246392154,gas,2011
74613,56041,56041,WY,Uinta County,106114215,gas,2011
74614,56043,56043,WY,Washakie County,1961277,gas,2011


In [25]:
df_3=df_2.pivot(index =["FIPS","Stabr","County_Name","geoid","Year"], 
columns = "Product",
 values = "Value")\
     .reset_index()\
         .rename_axis(columns = None)
df_3


,FIPS,Stabr,County_Name,geoid,Year,gas,oil
0,1003,AL,Baldwin County,1003,2000,72543902.0,138072.0
1,1003,AL,Baldwin County,1003,2001,98699994.0,134666.0
2,1003,AL,Baldwin County,1003,2002,107142655.0,138011.0
3,1003,AL,Baldwin County,1003,2003,101510068.0,127985.0
4,1003,AL,Baldwin County,1003,2004,90146850.0,130763.0
...,...,...,...,...,...,...,...
12980,56045,WY,Weston County,56045,2007,1989177.0,1115181.0
12981,56045,WY,Weston County,56045,2008,1688022.0,994900.0
12982,56045,WY,Weston County,56045,2009,1860619.0,907880.0
12983,56045,WY,Weston County,56045,2010,1948313.0,838371.0


lo ponemos todo en minusculas

In [26]:
df_final = df_3.\
    rename(columns = lambda column : column.lower()) \
    
df_final

,fips,stabr,county_name,geoid,year,gas,oil
0,1003,AL,Baldwin County,1003,2000,72543902.0,138072.0
1,1003,AL,Baldwin County,1003,2001,98699994.0,134666.0
2,1003,AL,Baldwin County,1003,2002,107142655.0,138011.0
3,1003,AL,Baldwin County,1003,2003,101510068.0,127985.0
4,1003,AL,Baldwin County,1003,2004,90146850.0,130763.0
...,...,...,...,...,...,...,...
12980,56045,WY,Weston County,56045,2007,1989177.0,1115181.0
12981,56045,WY,Weston County,56045,2008,1688022.0,994900.0
12982,56045,WY,Weston County,56045,2009,1860619.0,907880.0
12983,56045,WY,Weston County,56045,2010,1948313.0,838371.0


## Verificación de datos

In [15]:
df_final.sample(10)


,fips,stabr,county_name,geoid,year,gas,oil
8997,42063,PA,Indiana County,42063,2009,37109297.0,2631.0
1913,17189,IL,Washington County,17189,2002,NaN,244261.0
3544,21085,KY,Grayson County,21085,2011,NaN,842.0
2663,20081,KS,Haskell County,20081,2000,44418824.0,2397757.0
3762,21147,KY,McCreary County,21147,2008,224145.0,3017.0
12060,49041,UT,Sevier County,49041,2009,NaN,3040522.0
7512,39079,OH,Jackson County,39079,2002,7583.0,NaN
4472,22077,LA,Pointe Coupee Parish,22077,2005,59329189.0,765778.0
4161,22021,LA,Caldwell Parish,22021,2010,2121414.0,47.0
5856,28121,MS,Rankin County,28121,2007,161913447.0,78835.0


![imagen.png](attachment:ba93caa1-edf9-4492-85d5-6000bb3a1d69.png)

In [16]:
df_final.describe(include = 'all') 

,fips,stabr,county_name,geoid,year,gas,oil
count,12985.000000,12985,12985,12985.000000,12985.000000,1.107800e+04,1.147100e+04
unique,NaN,32,856,NaN,NaN,NaN,NaN
top,NaN,TX,Jackson County,NaN,NaN,NaN,NaN
freq,NaN,2631,105,NaN,NaN,NaN,NaN
mean,32113.044282,NaN,NaN,32113.044282,2005.558337,1.956302e+07,1.198280e+06
std,14839.581818,NaN,NaN,14839.581818,3.450000,6.058323e+07,6.446772e+06
min,1003.000000,NaN,NaN,1003.000000,2000.000000,1.000000e+00,1.000000e+00
25%,20193.000000,NaN,NaN,20193.000000,2003.000000,1.632258e+05,2.118650e+04
50%,31063.000000,NaN,NaN,31063.000000,2006.000000,1.752248e+06,1.575280e+05
75%,48087.000000,NaN,NaN,48087.000000,2009.000000,1.275291e+07,7.024365e+05


![imagen.png](attachment:7152f164-7f6a-4306-bc85-382fc8a221f4.png)

In [17]:
df_final.query("stabr == 'FL' and year == 2002") 

,fips,stabr,county_name,geoid,year,gas,oil
1352,12005,FL,Bay County,12005,2002,2909.0,4231.0
1355,12021,FL,Collier County,12021,2002,108457.0,925205.0
1368,12033,FL,Escambia County,12033,2002,1256509.0,654456.0
1380,12051,FL,Hendry County,12051,2002,18690.0,215672.0
1392,12071,FL,Lee County,12071,2002,12937.0,126354.0
1405,12113,FL,Santa Rosa County,12113,2002,2400355.0,1789777.0


![imagen.png](attachment:d50f0621-00d0-4450-9c78-02d4117772f9.png)

In [18]:
df_final.query("geoid == 54019 and year.between(2000,2005)",engine = 'python') 

,fips,stabr,county_name,geoid,year,gas,oil
12253,54019,WV,Fayette County,54019,2000,3436078.0,46.0
12254,54019,WV,Fayette County,54019,2001,3320104.0,32.0
12255,54019,WV,Fayette County,54019,2002,3310641.0,NaN
12256,54019,WV,Fayette County,54019,2003,3293598.0,82.0
12257,54019,WV,Fayette County,54019,2004,3156797.0,58.0
12258,54019,WV,Fayette County,54019,2005,2959227.0,82.0


![imagen.png](attachment:faa74637-5c5d-4cea-9777-a7279d1085c8.png)

In [19]:
( df_final
  .query("stabr in ('AL', 'FL', 'MT', 'PA', 'WV')") 
  .filter(["stabr", "gas", "oil"]) 
  .groupby("stabr").aggregate(["count", "sum"])
)

gas                 oil             
      count           sum count          sum
stabr                                       
AL      242  3.818252e+09   156   96737887.0
FL       63  6.723119e+07    63   31882546.0
MT      360  1.165759e+09   315  306083749.0
PA      377  3.721836e+09   265   25076196.0
WV      568  2.866954e+09   424   19218870.0

![imagen.png](attachment:64bb4561-3577-4ea8-91a0-c2631cb3fe88.png)